In [1]:
%reload_ext autoreload
%autoreload 2

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from app.system import System
from app.assistant import Assistant
from app.assistant_response_router import AssistantResponseRouter

from app.tools import PropertiesFilterTool, InfoExtractorTool
from app.assistant_response_router import AssistantResponseRouter
from app.agents import load_information_agent, create_booking_agent


In [3]:
system = System()
assistant = Assistant()
assistant_response_router = AssistantResponseRouter()



In [4]:
def handler(message, customer_number="99513718"):
    conversation = system.get_conversation(customer_number)
    print(f"Conversation: {conversation}")
    system.add_user_message(message, customer_number)
    print(f"New Conversation: {system.get_conversation(customer_number)}")
    assistant_response = assistant.run(message, conversation)
    print(f"Assistant Response: {assistant_response}")

    # assistant_response_router_input = system.get_conversation_string(customer_number)
    print(f"Assistant Response Router Input: {assistant_response}")
    next_action = assistant_response_router(assistant_response)
    print(f"Next Action: {next_action}")
    if "load_properties_information" in next_action["action"]:
        print(f"Loading properties...")
        properties_info_response = load_information_agent.run(input=system.get_conversation_string(customer_number))
        if properties_info_response != "":
            # system.add_assistant_message(properties_info_response, customer_number)
            print(f"Updating conversation with Properties loaded")
            system.add_system_message(properties_info_response["text"], customer_number)
            print(f"Getting new response from Assistant")
            new_assistant_response = assistant.get_completion_from_messages(system.get_conversation(customer_number))
            system.add_assistant_message(new_assistant_response, customer_number)
            return new_assistant_response
        else:
            print(f"Not information loaded from properties, continuew conversation.")
            system.add_assistant_message(assistant_response, customer_number)
            return assistant_response
    elif "pre_booking" in next_action["action"]:
        print(f"Creating pre booking...")
        properties_info_response = create_booking_agent.run(input=system.get_conversation_string(customer_number))
        print(f"Booking created: {properties_info_response}")
    else:
        print(f"Continue with conversation.")
        system.add_assistant_message(assistant_response, customer_number)
        return assistant_response



In [7]:
#handler("Hola", customer_number="99513718")
#handler("Me gustaria reservar una casa para 3 presonas por dos noches para el 20 de junio", customer_number="99513718")
handler("por 2 noches, no tengo presupuesto", customer_number="99513718")

#handler("Perfecto, me gustaria reservar la propiedad Sol", customer_number="99513718")
#handler("Agustin Sarasua, a@gmail.com, 099513718", customer_number="99513718")


#handler(" 3 personas para el 20 de junio por 2 noches", customer_number="99513718")
#handler("En mercedes, la mas economica", customer_number="99513718")

#handler("Perfecto, mer sirve", customer_number="99513718")
#handler("Aca te paso: Agustin Sarasua, a@gmail.com, 099513718", customer_number="99513718")



Conversation: [{'role': 'user', 'content': 'Me gustaria reservar una casa para 3 presonas por dos noches para el 20 de junio'}, {'role': 'assistant', 'content': '¡Hola! Claro, puedo ayudarte con eso. ¿Podrías decirme qué tipo de casa prefieres y cuál es tu presupuesto? Además, ¿podrías proporcionarme tu fecha de salida? ¡Gracias!'}]
New Conversation: [{'role': 'user', 'content': 'Me gustaria reservar una casa para 3 presonas por dos noches para el 20 de junio'}, {'role': 'assistant', 'content': '¡Hola! Claro, puedo ayudarte con eso. ¿Podrías decirme qué tipo de casa prefieres y cuál es tu presupuesto? Además, ¿podrías proporcionarme tu fecha de salida? ¡Gracias!'}, {'role': 'user', 'content': 'por 2 noches, no tengo presupuesto'}]
Assistant Response: ¡Hola! Entendido, para verificar la disponibilidad, necesito saber cuántas personas serán y las fechas de entrada y salida. ¿Podrías proporcionarme esa información, por favor? ¡Gracias!
Assistant Response Router Input: ¡Hola! Entendido, pa

'¡Hola! Entendido, para verificar la disponibilidad, necesito saber cuántas personas serán y las fechas de entrada y salida. ¿Podrías proporcionarme esa información, por favor? ¡Gracias!'

In [21]:

# info_extractor_tool = InfoExtractorTool()
# info_extractor_tool_result = info_extractor_tool.run({"query": "Hola, me gustaria alquilar una casa para el 2 de setiembre para 3 personas por 2 noches"})
# info_extractor_tool_result
# # models = get_models_tool.run({"query_params": {"search": "gpt-j"}})


# properties_filter_tool = PropertiesFilterTool()
# properties_filter_tool.run({'checkin_date': '2023-06-20',
#  'checkout_date': '2023-06-23',
#  'num_nights': '3',
#  'num_guests': '3'})


In [8]:

system.get_conversation("99513718")


[{'role': 'user',
  'content': 'Me gustaria reservar una casa para 3 presonas por dos noches para el 20 de junio'},
 {'role': 'assistant',
  'content': '¡Hola! Claro, puedo ayudarte con eso. ¿Podrías decirme qué tipo de casa prefieres y cuál es tu presupuesto? Además, ¿podrías proporcionarme tu fecha de salida? ¡Gracias!'},
 {'role': 'user', 'content': 'por 2 noches, no tengo presupuesto'},
 {'role': 'assistant',
  'content': '¡Hola! Entendido, para verificar la disponibilidad, necesito saber cuántas personas serán y las fechas de entrada y salida. ¿Podrías proporcionarme esa información, por favor? ¡Gracias!'}]